# Document Chracterizzation


## Import of relevant libraries 

In [1]:
import os
import shutil
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

## Dataset import

In [2]:
## import the CSV file with the labels

data_input = pd.read_csv('Document Characterisation Data Dump.csv', sep=',')
print(len(data_input["URL"]))
data_input


129


,URL,CLASS
0,https://issafrica.org/iss-today/time-to-tackle...,Analytical Report
1,http://www.understandingwar.org/backgrounder/r...,Analytical Report
2,http://www.understandingwar.org/backgrounder/r...,Analytical Report
3,http://www.understandingwar.org/backgrounder/w...,Analytical Report
4,http://www.understandingwar.org/backgrounder/r...,Analytical Report
...,...,...
124,https://rewardsforjustice.net/english/muhammad...,Profile Report
125,https://rewardsforjustice.net/english/abdelkar...,Profile Report
126,https://rewardsforjustice.net/english/muhammad...,Profile Report
127,https://rewardsforjustice.net/english/jamal_ra...,Profile Report


In [8]:
def scraper(input):
    content = []
    for i in range(len(input["URL"])):
        url = input["URL"][i]
        text = url

        if not (text.startswith('https://') or text.startswith('http://')):
            text += ' Link not good'
            content.append(text) 
            continue
        
        page = requests.get(url)
        
        if page.status_code != 200:
            text += ' Page not able to find'

        else:
            soup = BeautifulSoup(page.content, 'html.parser')
            for p in soup.find_all('p'):
                text = text + ' ' + p.text
        
        content.append(text) 


    return content

In [19]:
data = scraper(data_input)
data_input['Content'] = data
data_input

,URL,CLASS,Content
0,https://issafrica.org/iss-today/time-to-tackle...,Analytical Report,https://issafrica.org/iss-today/time-to-tackle...
1,http://www.understandingwar.org/backgrounder/r...,Analytical Report,http://www.understandingwar.org/backgrounder/r...
2,http://www.understandingwar.org/backgrounder/r...,Analytical Report,http://www.understandingwar.org/backgrounder/r...
3,http://www.understandingwar.org/backgrounder/w...,Analytical Report,http://www.understandingwar.org/backgrounder/w...
4,http://www.understandingwar.org/backgrounder/r...,Analytical Report,http://www.understandingwar.org/backgrounder/r...
...,...,...,...
124,https://rewardsforjustice.net/english/muhammad...,Profile Report,https://rewardsforjustice.net/english/muhammad...
125,https://rewardsforjustice.net/english/abdelkar...,Profile Report,https://rewardsforjustice.net/english/abdelkar...
126,https://rewardsforjustice.net/english/muhammad...,Profile Report,https://rewardsforjustice.net/english/muhammad...
127,https://rewardsforjustice.net/english/jamal_ra...,Profile Report,https://rewardsforjustice.net/english/jamal_ra...


## Experimental Data Analysis 

Pre trained BERT models

https://pypi.org/project/pytorch-pretrained-bert/

https://pytorch.org/hub/huggingface_pytorch-transformers/

https://medium.com/@aniruddha.choudhury94/part-2-bert-fine-tuning-tutorial-with-pytorch-for-text-classification-on-the-corpus-of-linguistic-18057ce330e1
